# Оптимизационные задачи в ритейле

Привет, Habr! На связи отдел аналитики данных X5 Tech.

Сегодня мы поговорим об очень интересном разделе прикладной математики — оптимизации.

Цели данной статьи:
* рассказать про задачи в ритейле, которые могут решаться методами оптимизации,
* продемонстрировать, как модельная задача ценообразования решается пакетами [__Pyomo__](http://www.pyomo.org/) и [__Scipy__](https://scipy.org/),
* сравнить производительность солверов PYOMO и Scipy на примере поставленной задачи.

Так как данная тема достаточно обширна, то помимо данной статьи (статья 1) мы планируем написать ещё две:

    1) Статья 2: Сравнение open-source солверов на примере задачи ритейла.
    2) Статья 3: Решение модельной задачи ценообразования оптимизаторами в различных постановках.


## Примеры задач


Практически каждый человек ежедневно решает оптимизационные задачи даже не задумываясь об этом. Пара примеров:

**Закупка.**
Как правило, мы хотим минимизировать наши расходы для приобретения необходимых товаров, но при условии, чтобы эти товары были максимально полезны.
Полезность здесь у каждого своя:
для одних она определяется количеством растительных жиров,
для других — минимальной суммарной стоимостью корзины,
для третьих — наличием привычных товаров, и тд.

**Отпуск.**
Во время ограниченного отпуска мы хотим распределить свой маршрут так,
чтобы посетить всё, что запланировали с минимальными временными затратами на дорогу, и при этом не забыть позагорать на пляже.

Бизнес требования часто приводят к задаче многокритериальной оптимизации и задаче оптимального управления.
В таких постановках решения, найденные методами оптимизации, чрезвычайно полезны для принятия решений.
Приведём для иллюстрации несколько задач из области ритейла.

**Ценообразование.**
Поиск оптимальной конфигурации цен с учетом ценового позиционирования, допустимых ценовых диапазонов
для каждого товара и набора бизнес-ограничений.
Цены должны максимизировать суммарный доход, а прибыль быть не ниже на заданного уровня.

**Оптимальное распределение маркетингового бюджета.**
Реализовать максимально эффективно выделенный на маркетинговые активности бюджет.
Есть несколько каналов для рекламных акций, цель — максимально выгодно инвестировать бюджет,
чтобы суммарный доход со всех коммуникаций был максимален.
Также необходимо учесть бизнес ограничения на нагрузку каждого канала + частоту взаимодействия.

**Планирование ассортимента.**
Подобрать ассортимент товаров так, чтобы максимизировать оборот с учетом полочного пространства и других характеристик магазина.

**Закупка товаров.**
Задача — распределить бюджет, выделенный на закупки, для поддержания товарооборота, достаточного уровня сервиса,
и при этом достигнуть определенных финансовых показателей под выделенный бюджет на закупки.

**Логистика.**
Задача — найти оптимальный график доставки продуктов с учётом вместимости грузовиков, затрат на логистику и тд.


## Общая постановка задачи и её разновидности


Прежде всего рассмотрим постановку задачи в общем виде:

$x$ - вектор размерности $n$, $x \in X$ - допустимое множество значений этих переменных.

$f(x) \to \min(\max)$, $f(\cdot)$ - целевая функция

$g_i(x) \leqslant 0, \ i=1..m$ - ограничения вида неравенств

$h_i(x) = 0, \ j=1..k$ - ограничения вида равенств

Исходя из практики можно разложить данную постановку на несколько классов в зависимости от вида целевой функции, ограничений и $X$:

* __Безусловная оптимизация__ $g_i(x), h_j(x)$ - отсутствуют, $X = \mathbb{R}^n$;

* __LP__ (linear programming) - линейное программирование. $f(x), g_i(x), h_j(x)$ - линейные функции, $X = \mathbb{R}_+^n$;

* __MILP__ (mixed integer linear programming) - смешанное целочисленное линейное программирование, это задача LP в которой только часть переменных являются целочисленными;

* __NLP__ (nonlinear programming) - нелинейное программирование, возникает когда хотя бы одна из функций $f(x),\ g_i(x),\ h_j(x)$ нелинейна;

* __MINLP__ (mixed integer nonlinear programming) - смешанное целочисленное нелинейное программирование,
возникает как и в MILP, когда часть переменных принимает целочисленные значения;

__NLP__ в свою очередь можно подробить еще на множество разных классов в зависимости от вида нелинейности и выпуклости.

Подробнее о том, как получаются различные виды задачи ценообразования исходя из бизнес-формулировки, мы будем говорить в статье 3.

## Оптимизация модельной задачи ценообразования

Разобравшись с разными разновидностями постановок оптимизационных задач, перейдём к построению модели.

Предположим, что для товара $i$ известно значение эластичности $E_{i}$, а спрос задаётся следующей зависимостью:

\begin{equation}
Q_{i}(P_{i}) = Q_{0, i} \exp\bigg(E_{i} \cdot \bigg(\frac{P_{i}}{P_{0, i}} - 1\bigg)\bigg).
\end{equation}

Введём обозначения:
* $n$ - количество товаров,
* $C_{i}$ - себестоимость i-го товара,
* $Q_{i}, Q_{0, i}$ - спрос по новой $P_{i}$ и текущей $P_{0, i}$ ценам, соответственно,
* $x_{i}=\cfrac{P_{i}}{P_{0, i}}$.

Задача — найти такой набор новых цен, чтобы:
 * максимизировать оборот со всех товаров,
 * общая прибыль осталась на прежнем уровне,
 * цены лежали в заданных границах (индекс ***l*** и ***u*** - для нижней и верхней, соответственно).

Тогда оптимизационную задачу можно записать следующей системой:

\begin{cases}
\sum_{i=1}^{n} P_i \cdot Q_i(x_i) \to \max_{x},\\
\\
\sum_{i=1}^{n} (P_i - C_i) \cdot Q_i(x_i) \geqslant \sum_{i=1}^{n} (P_{0, i} - C_i) \cdot Q_ {0, i},\\
\\
x_i \in [x_i^l, x_i^u], \ i=1..n\\
\end{cases}

## Реализация модели в PYOMO и Scipy

Для демонстрации решения необходимы данные.

Реальные данные мы использовать не можем, поэтому будем генерировать их из случайных распределений (функция generate_base_data),
исходя из наших представлений о возможных значениях величин в фуд-ритейле.

<details>
<summary> Пример данных для NLP постановки: </summary>
<img src="images/a11_data.png" width="300" align="left"/>
</details>


Scipy и PYOMO имеют разные интерфейсы, чтобы как-то унифицировать работу с ними,
будем наследоваться от базового класса, [код класса](https://github.com/mbudylin/OptimizersArticle/blob/main/optimizers/optimizers.py).

Методы, которые необходимо реализовать для каждого солвера:

* init_objective - задание целевой функции,
* init_constraints - статические ограничения,
* add_constraints - динамические ограничения,
* solve - поиск оптимального решения.

Опишем далее отличия в реализации этих методов в Scipy и PYOMO.
Другие примеры из официальной документации можно найти
[здесь, для pyomo](https://pyomo.readthedocs.io/en/stable/working_models.html) и
[здесь, для scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html).

### Метод init_objective

В Scipy оптимизатор работает в режиме минимизации, поэтому суммарный оборот берём со знаком "-".

В PYOMO функцию пересчёта суммарного оборота необходимо передать в переменную expr объекта pyo.Objective.

```python
# SCIPY
def objective(x):
    return -sum(self.P * x * self.Q * self._el(self.E, x))

# PYOMO
objective = sum(self.P[i] * self.model.x[i] * self.Q[i] * self._el(i) for i in range(self.N))
self.model.obj = pyo.Objective(expr=objective, sense=pyo.maximize)
```


### Методы init_constraints и add_constraints

Ограничение на суммарную прибыль задаётся в методе init_constraints.

Для Scipy ограничения передаются через NonlinearConstraint или LinearConstraint().

Для PYOMO ограничения передаются через pyo.Constraint().

```python
# SCIPY
def init_constraints(self):
    def con_mrg(x):
        m = sum((self.P * x - self.C) * self.Q * self._el(self.E, x))
        return m
    constr = NonlinearConstraint(con_mrg, self.m_min, np.inf)
    self.constraints.append(constr)

# PYOMO
con_mrg_expr = sum((self.P[i] * self.model.x[i] - self.C[i]) * self.Q[i] * self._el(i)
                   for i in range(self.N)) >= self.m_min
self.model.con_mrg = pyo.Constraint(rule=con_mrg_expr)
```


### Метод solve

В Scipy задача оптимизации запускается через minimize, а в PYOMO предварительно
через проинициализированный объект SolverFactory методом solve.

```python
# SCIPY
result = minimize(self.obj,
                  self.x0,
                  method=solver,
                  constraints=self.constraints,
                  options=options)

# PYOMO
solver = pyo.SolverFactory(solver, tee=False)
result = solver.solve(self.model)
```


## Расчёт и результаты

Так как оптимизаторы устанавливаются отдельно от python, то для получения аналогичных результатов можно
воспользоваться Dockerfile, предварительно настроив контейнер, как описано в README проекта.


Чтобы запустить расчёты, нужно выполнить команду:

```console
python runner_simple.py -m pyomo
python runner_simple.py -m scipy
```

Для сравнения расчётов достаточно выполнить скрипт команду:

```console
python runner_simple.py -m compare
```

Зависимость длительности поиска оптимального решения от размера задачи представлена на графике ниже.
Из графика можно сделать вывод, что PYOMO с солвером ipopt значительно превосходит Scipy с COBYLA при N > 50.

<img src="./images/time_solve_simple.png" width="900" align="c"/>

## Заключение

В данной статье мы:
* рассмотрели, какие задачи оптимизации бывают на практике,
* показали, как можно решать задачу ценообразования с помощью PYOMO,
* сравнили производительность солверов PYOMO и Scipy.

В следующей стать более детально обсудим другие open-source солверы для python, их различия в реализации и производительности.

Над статьёй работали Антон Денисов, Михаил Будылин.
